In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
import pandas as pd
import numpy as np
from functools import reduce
from pyspark.sql.types import FloatType

In [2]:
spark = (
    SparkSession.builder.appName("preprocessing of taxi data")
    .config("spark.sql.repl.eagerEval.enabled", True) 
    .config("spark.sql.parquet.cacheMetadata", "true")
    .config("spark.sql.session.timeZone", "Etc/UTC")
    .config("spark.driver.memory", "15g")
    .getOrCreate()
)

22/10/01 17:02:15 WARN Utils: Your hostname, Haiyangs-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 10.12.175.225 instead (on interface en0)
22/10/01 17:02:15 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/10/01 17:02:16 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
age_sdf = spark.read.option("header", True).csv("2016_age.csv")
education_sdf = spark.read.option("header", True).csv("2016_education.csv")

In [4]:
columns = ["18_24", "25_34", "35_44", 
           "45_54", "55_64", "65_plus", 
           "sub_total", "postcode"]

age_group = columns[:-1]

age_sdf = age_sdf.withColumn(columns[6], reduce(lambda x,y:x+y, [F.col(str(x)) for x in range(18, 116)])) \
                 .withColumn(columns[0], reduce(lambda x,y:x+y, [F.col(str(x)) for x in range(18, 25)]) / F.col(columns[6])) \
                 .withColumn(columns[1], reduce(lambda x,y:x+y, [F.col(str(x)) for x in range(25, 35)]) / F.col(columns[6])) \
                 .withColumn(columns[2], reduce(lambda x,y:x+y, [F.col(str(x)) for x in range(35, 45)]) / F.col(columns[6])) \
                 .withColumn(columns[3], reduce(lambda x,y:x+y, [F.col(str(x)) for x in range(45, 55)]) / F.col(columns[6])) \
                 .withColumn(columns[4], reduce(lambda x,y:x+y, [F.col(str(x)) for x in range(55, 65)]) / F.col(columns[6])) \
                 .withColumn(columns[5], reduce(lambda x,y:x+y, [F.col(str(x)) for x in range(65, 116)]) / F.col(columns[6])) \
                 .withColumn(columns[7], F.regexp_extract('AGEP Age', r'\d+', 0)) \
                 .select(*columns)
                
age_sdf.limit(5)

22/10/01 17:02:23 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


18_24,25_34,35_44,45_54,55_64,65_plus,sub_total,postcode
0.2166044129539066,0.41118256382610036,0.1598059224459933,0.08290654241595749,0.06426893603912356,0.06523162231891871,25969.0,2000
0.9347107438016529,0.045454545454545456,0.00743801652892562,0.004958677685950...,0.00743801652892562,0.0,1210.0,2006
0.39983285577841454,0.3390639923591213,0.11926934097421203,0.05742597898758357,0.04035339063992359,0.04405444126074499,8376.0,2007
0.39916003931730853,0.3594853006880529,0.10481637029756054,0.061388615852023945,0.042623536770619246,0.032526137074434816,11191.0,2008
0.1143575515866135,0.3546904803190567,0.20469915033813074,0.11938616264955783,0.10499393098664817,0.10187272411999307,11534.0,2009


In [5]:
student_sdf = education_sdf.withColumn("postcode", F.regexp_extract("postcode", r'\d+', 0)) \
                           .withColumnRenamed("total", "total_parttime_or_fulltime_students") \
                           .select("postcode", "total_parttime_or_fulltime_students")

student_sdf.limit(5)

postcode,total_parttime_or_fulltime_students
2000,8101
2006,1332
2007,4008
2008,5147
2009,1726


In [6]:
income_sdf = spark.read.option("header", True).csv("income_cleaned.csv") \
                                              .select("postcode", "median_salary")
income_sdf.limit(5)

postcode,median_salary
2000,575.0
2006,75.0
2007,350.0
2008,575.0
2009,1125.0


In [7]:
def mean(column_name):
   return F.mean(column_name).alias("mean_" + column_name)

def processing_pipeline(dlist):
    for data in dlist:
        # Join persona data
        data = data.join(income_sdf, "postcode", "left") \
                   .join(student_sdf, "postcode", "left") \
                   .join(age_sdf, "postcode", "left")

        # remove rows contain null in persona
        data = data.filter(F.col("median_salary").isNotNull() \
                         | F.col("sub_total").isNotNull() \
                         | F.col("total_parttime_or_fulltime_students").isNotNull())

        # take the mean of all persona
        agg_data = data.groupBy("merchant_abn") \
                       .agg(
                            mean("median_salary"),
                            mean("total_parttime_or_fulltime_students"),
                            *[mean(name) for name in age_group]
                            )
        yield agg_data
        

In [8]:
transaction_sdf = spark.read.parquet("../data/curated/transactions_withoutfraud") \
                            .drop('user_id2',
                                  'order_datetime2',
                                  'fraud rate',
                                  'fraud2',
                                  'user_id3',
                                  'order_datetime3',
                                  'fraud_probability',
                                  'fraud3')

# training data is the partition before the first fortnight of 2022
train_transaction_sdf = transaction_sdf.where(F.col("order_datetime") <= "2022-1-16")


data_list = [transaction_sdf, train_transaction_sdf]

data_list = list(processing_pipeline(data_list))

data_list[0].limit(5)

merchant_abn,mean_median_salary,mean_total_parttime_or_fulltime_students,mean_18_24,mean_25_34,mean_35_44,mean_45_54,mean_55_64,mean_65_plus,mean_sub_total
34440496342,665.3571428571429,726.0285714285715,0.09616178597603435,0.14638469969797188,0.15608870164698682,0.1735831056885002,0.20771864696238573,0.220063060028121,8046.771428571428
15613631617,647.4765258215963,635.1502347417841,0.09355564743741474,0.14970744876770364,0.15654891609964974,0.17981347180449017,0.1928799103803407,0.22749460551040104,6140.830985915493
83412691377,665.4689608636977,658.808439383791,0.10427259065156969,0.15142024706844687,0.15652496789966566,0.18378714204466093,0.1838697186090033,0.22012533372665358,6706.924426450742
24406529929,661.9674887892377,613.1066666666667,0.09703010735768407,0.1508463018793074,0.15552580740901656,0.18518170229690945,0.18534575633732567,0.22607032471975688,6236.961883408072
73256306726,660.6919642857143,703.9039145907474,0.09931809556504945,0.14988948980853775,0.1557895084700852,0.187349726576246,0.1821697535401777,0.22548342603990384,7043.998214285714


In [9]:
def compute_persona_score(dlist, filenames):
    for data, filename in zip(dlist, filenames):
        data = data.withColumn("pop_index",
                                0.26*F.col("mean_18_24") + 
                                0.35*F.col("mean_25_34") + 
                                0.20*F.col("mean_35_44") + 
                                0.12*F.col("mean_45_54") + 
                                0.05*F.col("mean_55_64") +
                                0.01*F.col("mean_65_plus")
                                )
                                
        data = data.select("merchant_abn",
                            "mean_median_salary", 
                            "mean_total_parttime_or_fulltime_students",
                            "pop_index",
                            "mean_sub_total")

        for col in data.columns[1:]:
            max_ = data.agg({col:"max"}).collect()[0][0]
            min_ = data.agg({col:"min"}).collect()[0][0]
            data = data.withColumn(col, normalization_(F.col(col), F.lit(max_), F.lit(min_)))
            data = data.withColumnRenamed(col, "normalized_"+col)
        

        data = data.withColumn("persona_score",
                               F.col("normalized_mean_median_salary") *
                               F.col("normalized_mean_total_parttime_or_fulltime_students") *
                               F.col("normalized_pop_index") *
                               F.col("normalized_mean_sub_total")
                               )
        
        data.select("merchant_abn", "persona_score").toPandas().to_csv(filename, index=False)
        
        print("%s is done" % filename)

        yield data
        

@F.udf(returnType=FloatType())
def normalization_(val, col_max, col_min):
    val = float(val)
    col_max = float(col_max)
    col_min = float(col_min)
    return (val - col_min) / (col_max - col_min)


In [10]:
filenames = ["full_persona_score.csv", "trained_persona_score.csv"]
list(compute_persona_score(data_list, filenames))[0].limit(5)

full_persona_score.csv is done


trained_persona_score.csv is done
